Created on 8 Aug 2021
한터 데이터저널리즘 3기 웹크롤링

ㅇ과제(목표): 
[한국기자협회 이달의기자상](http://www.journalist.or.kr/news/section1.html?p_num=2) 모든 수상 목록 가져오기 

@author: Osung

In [ ]:
#라이브러리 불러오기. 함수들의 도서관
from selenium import webdriver
from selenium.common.exceptions import UnexpectedAlertPresentException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait

셀레니움 selenium이 설치되어 있지 않다는 에러가 뜰 것입니다  
주피터노트북에선 파이썬의 문법은 아니지만 커맨드창 명령어를 입력해 없는 라이브러리를 설치할 수 있습니다  
입력 문법은 아래와 같습니다

In [ ]:
pip install selenium

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
pip install bs4

In [ ]:
browser = webdriver.Chrome()

In [ ]:
# web driver를 인식 못하는 경우를 위한 예비 코드
chrome_path = r'C:\Users\hannews\Documents\2021-07_한터_데이터 저널리즘_3기\chromedriver.exe'
browser = webdriver.Chrome(executable_path=chrome_path) #웹브라우저 파일을 찾지 못한다는 에러에 대한 ts

코딩에 앞서 중요한 것은 '문제의 이해'(Understanding the problem)  
먼저 스크랩할 목표인 [대상 페이지](http://www.journalist.or.kr/news/section1.html?p_num=2)를 찬찬히 살펴봅니다  
일반적으로 다음과 같은 사항을 체크할 필요가 있습니다.  
* 스크랩 대상 페이지는 어떻게 구성 되어 있나? 
* 목표로 하는 정보 수집을 충족하기 위해 어떤 요소 elements를 수집해야 하는가?
* 이에 대한 자동화된 수집을 수행할 때 특별한 제약 사항은 없겠는가?
* 위 조건은 사이트의 수집 대상인 모든 페이지들에 동일한가?
* 수집 대상 페이지의 주소들에 일정한 규칙이 있는가? (누락 페이지는 없는가)

문제를 이해하고 난 다음에는 해결책을 '디자인'해야 합니다  
디자인의 요점은 해결 과정을 잘게 나눈 뒤  
각 과정을 잘 꿰어  
최종 결과물을 완성하는 것입니다  

그리고 잘게 나눈 각 과정은 실제 '코딩'한 뒤  
실행 결과를 보고  
잘못 작동하는 점 등을 확인해  
다시 업데이트하는 절차('되돌아보기')를 거칩니다  
인터프리터 언어는 이런 사이클을 수행하기 쉽게 합니다

In [ ]:
# 한 페이지에서 한 요소를 가져오는 것부터 출발. 테스트 페이지
a = 'https://www.journalist.or.kr/mybbs/bbs.html?mode=view&bbs_code=bbs_4&cate=&page=1&search=&keyword=&type=adm&bbs_no=1'

In [ ]:
browser.get(a)

In [ ]:
'''
함수는 코딩에서 매우 중요한 영역을 차지하고 있습니다
아래는 브라우저에 로딩한 페이지의 html을 쓰기 좋게 해체(파싱)하는 함수를 정의하고 있습니다 
별도 함수로 코딩하는 이유는 이 기능을 원하는 때마다 편하게 불러 쓰기 위함입니다
'''
def parser(url):
    try: 
        browser.get(url)
    except UnexpectedAlertPresentException:  #페이지의 예외적인 경우를 위한 코드
        time.sleep(1)
        alert = browser.switch_to_alert()
        time.sleep(1)
        print(alert.text)
        alert.accept()
        WebDriverWait(browser, 5)
    HTML = browser.execute_script("return document.body.innerHTML")
    browser.implicitly_wait(3) #3초를 기다려 줍니다
    return BeautifulSoup(HTML, 'html.parser')

In [ ]:
# 잘 작동하는지 볼까요?
soup = parser(a)
soup

In [ ]:
'''
파싱이 잘 된다면 첫번째 요소를 수집해 봅시다
게시글의 제목(몇회 수상작인지 표시)을 가져와 봅시다
요소를 결정하는 코드를 짜는 이 부분에서
HTML과 CSS Selector에 대한 이해가 필요합니다
'''
title = soup.find("p", class_="subcnt")
print(title)

In [ ]:
#활용에 불필요한 html 코드가 붙어있습니다
#필요한 텍스트만 추출
title.text.strip()

In [ ]:
'''
잘 작동하는 것 같습니다
그런데 이 페이지에서만 운 좋게 작동하는 것은 아닐까요?
시작 전에 눈과 머리로 확인은 하였지만
실제 잘 작동하는지 다음 페이지에서도 실험해 봅니다
'''
soup2 = parser('https://www.journalist.or.kr/mybbs/bbs.html?mode=view&bbs_code=bbs_4&cate=&page=1&search=&keyword=&type=adm&bbs_no=2')
title2 = soup2.find("p", class_="subcnt")
title2.text.strip()

In [ ]:
# 기대와 달리 더 오래된 수상 정보가 두번째에 등장함도 주의하십시오
# 하나 더 실험해 봅시다. 임의의 최근 페이지에서도 작 작동할까요?
soup3 = parser('https://www.journalist.or.kr/mybbs/bbs.html?mode=view&bbs_code=bbs_4&cate=&page=1&search=&keyword=&type=adm&bbs_no=30907')
title3 = soup3.find("p", class_="subcnt")
title3.text.strip()

In [ ]:
'''
제목은 잘 작동하는 것 같습니다
그럼 메인 타깃이라 할 목록을 가져와 봅시다
역시 HTML과 CSS에 대한 이해를 바탕으로
효과적이고 창의적으로 Selector에 대한 코드를 짤 필요가 있습니다
'''
body = soup3.select(".cnt tr td p")
body

In [ ]:
# 역시 불필요한 부분을 제거하는 적절한 가공이 필요해 보이네요
content = str()
for element in body:
    content = content + element.text.strip() + '\n'
print(content)

In [ ]:
# 역시 다른 페이지에 대한 테스트를 수행합니다
body2 = soup2.select(".cnt tr td p")
content2 = str()
for element in body2:
    content2 = content2 + element.text.strip() + '\n'
print(content2)

In [ ]:
# 작동하지 않는 경우 문제를 확인해서 해결해야 합니다
body2 = soup2.select(".cnt tr td")
content2 = str()
for element in body2:
    content2 = content2 + element.text.strip() + '\n'
print(content2)

작은 단위의 작동을 확인하고 난 뒤  
전체 단위의 디자인을 계획합니다

In [ ]:
'''
이 과정에서 전형적인 문제는 페이지 누락입니다. 
누락 페이지를 일일이 확인하고 제외하는 것은 번거로운 일입니다.
중간에 누락 페이지 경우 스크랩을 수행하지 않고 건너뛰도록 한다면
이 문제를 수월하게 다룰 수 있을 것입니다
'''
temp = parser('https://www.journalist.or.kr/mybbs/bbs.html?mode=view&bbs_code=bbs_4&cate=&page=1&search=&keyword=&type=adm&bbs_no=30906')

In [ ]:
'''
이런 문제를 다룰 수 있게 parser 함수를 업그레이드할 필요가 있습니다
# 참조: https://liveyourit.tistory.com/38
'''
def parser(url):
    browser.get(url)
    time.sleep(5)
    try: 
        alert = browser.switch_to.alert
        time.sleep(2)
        print(alert.text)
        time.sleep(2)
        alert.accept()
        return None
    except:
        HTML = browser.execute_script("return document.body.innerHTML")
        element = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "subcnt")))
        return BeautifulSoup(HTML, 'html.parser')
#         except TimeoutException:
#             print("Timeout")
#             return None

In [ ]:
#이제 최종 결과물을 만들어 봅시다
LatestNum = 100 #bbs 게시물 1번부터 이 번호까지 게시물 수집
수상작 = str() #최종 산출 데이터 수집하는 변수

for i in range(LatestNum):
    url = 'https://www.journalist.or.kr/mybbs/bbs.html?mode=view&bbs_code=bbs_4&cate=&page=1&search=&keyword=&type=adm&bbs_no='+str(i+1)
    print(url)
    try:
        soup = parser(url)
    except TimeoutException:
        print("Timeout")
        continue
    
    if soup:
        title = soup.find("p", class_="subcnt")
        title2 = title.text.strip()
        body = soup.select(".cnt tr td")
        content = str()
        for element in body:
            content = content + element.text.strip() + '\n'

        수상작 = 수상작 + '■■' + str(title2) + '\n' + str(content) + '\n\n' #출력 양식: ■■로 회별 구분(괄호에 시점) / 한줄 띈뒤 내용    
    else:
        수상작 = 수상작 + '[N/A]' + url + '\n\n'

In [ ]:
수상작

## 결과물 Output 생산  
웹은 바다와 같아서 예측하기 어려운 상황에 따라 웹스크래핑도 많은 영향을 받습니다  
수집의 신뢰성을 높이려다 보면 효율이 떨어질(들어가는 시간이 늘어날) 수 있습니다  
효율을 높이다 보면 수집 데이터의 신뢰성이 위험해 질 수 있습니다  
지속적인 '되돌아보기'가 필요하겠습니다  
만족할만한 결과물이 완성되었다면 파이썬 코드의 영역에서 건져서  
다른 환경에서 쓸 수 있는 파일로 쓰는 과정이 필요합니다  
다양한 포맷으로 쓰기가 가능하며 여기선 가장 단순한 사례를 보겠습니다

In [ ]:
with open("이달의기자상 목록.txt", "w", encoding="utf-8-sig") as f:
    f.write(수상작)

## 되돌아보기
이후 코드를 업그레이드 할 여러 여지가 있습니다